# Temporal Difference (TD)
* 기존 MC Learning은 Episode가 끝나야 Evaluation을 하고, Improvement를 할 수 있다.
* 하지만, 이러한 단점에 비해 Environment(State transition probability와 Expected Reward)를 몰라도 된다는 장점이 있다.
* 여기서 Episode가 안 끝나도 Dynamic Programming처럼 Update를 할 수 있다면 어떨까?
    * 여기서 말하는 Update는 Improvement가 아니라 V(S)에 대한 Update라서 Evaluation을 의미한다.
* 해당 아이디어를 기반으로 제안된 방법론이 Temporal Difference이다.
* * *
* 기존 MC는 Incremental Mean을 사용해서 $V(S_t) \leftarrow V(S_t) + \alpha(G_t - V(S_t))$을 통해 Evaluation 되었다.
* $G_t$를 Episode가 다 끝나야 구할 수 있었는데, 이를 DP를 사용해서 $r_{t+1}+\gamma V(s')$로 바꾼다면 가능하다. 
$$ V(S_t) \leftarrow V(S_t) + \alpha(r_{t+1}+\gamma V(s') - V(S_t)) $$

In [7]:
import numpy as np

In [8]:
class GridWorld:
    '''
    Environment, Grid 4x4
    '''
    def __init__(self):
        self.agent_pos = {
            'y': 0,
            'x': 0
        }
        
        self.goal_pos = {
            'y': 3,
            'x': 3
        }
        
        self.y_min, self.x_min, self.y_max, self.x_max = 0, 0, 3, 3
        
        # 해당 코드에서는 State가 4x4 matrix 그 자체라 보면된다.
        self.state = np.zeros([4, 4]) # 4x4 Grid
        # 그래서 아래는 Initial state라고 보면 됨.
        self.state[self.agent_pos['y'], self.agent_pos['x']] = 1
        
        # 모든 state에 대한 저장
        self.state_space = list()
        for y in range(4):
            for x in range(4):
                state = np.zeros([4, 4])
                state[y, x] = 1
                self.state_space.append(state)
        
        self.action_space = [0, 1, 2, 3] # U, D, L, R
        self.gamma = 0.9
        
    
    def reset(self):
        self.agent_pos = {
            'y': 0,
            'x': 0
        }
        self.state = np.zeros([4, 4])
        self.state[self.agent_pos['y'], self.agent_pos['x']] = 1
        
        return self.state
    
    def step(self, action):
        '''
        Args:
            action: [0, 3] 범위의 int 값으로 어떤 Action을 취하는지 의미한다.
        Return:
            reward: s->a->s'에 대한 Reward
            self.state: (list) 다음 state(4x4)
            done: (bool) Goal state에 도달했는가?
        '''
        
        # Action에 따라 state를 이동하면서 Boundary를 넘지 않도록 한다.
        if action == 0:
            self.agent_pos['y'] = max(self.agent_pos['y']-1, self.y_min)
        elif action == 1:
            self.agent_pos['y'] = min(self.agent_pos['y']+1, self.y_max)
        elif action == 2:
            self.agent_pos['x'] = max(self.agent_pos['x']-1, self.x_min)
        elif action == 3:
            self.agent_pos['x'] = min(self.agent_pos['x']+1, self.x_max)
        else:
            raise AssertionError("Invalid Action")
        
        # State transition(self.state 갱신) 및 Get reward
        prev_state = self.state
        self.state = np.zeros([4, 4])
        self.state[self.agent_pos['y'], self.agent_pos['x']] = 1
        reward = self.reward(prev_state, action, self.state)
        
        # Episode done?
        done = True if (self.agent_pos == self.goal_pos) else False
        
        return reward, self.state, done
    
    def reward(self, s, a, s_next):
        reward = 0
        y, x = np.where(s == 1)
        y_next, x_next = np.where(s_next == 1)
        
        # State transition 했을 때, Goal state인가?
        if (
            (y_next == self.goal_pos['y'] and x_next == self.goal_pos['x']) and
            (y != self.goal_pos['y'] or x != self.goal_pos['x'])
            ):
            reward = 10
            
        return reward

    def get_state_index(self, state_space, state):
        # State에 해당하는 index가 무엇인가?
        for i_s, s in enumerate(state_space):
            if (s == state).all():
                return i_s
        raise AssertionError("Couldn\'t find the state from the state space")
    
    def exploring_start(self):
        # Initial State를 랜덤하게 선택
        while True:
            y_random = np.random.randint(4)
            x_random = np.random.randint(4)
            self.agent_pos = {
                'y': y_random,
                'x': x_random
            }
            if self.agent_pos != self.goal_pos:
                break
        
        self.state = np.zeros([4, 4])
        self.state[self.agent_pos['y'], self.agent_pos['x']] = 1
        return self.state

In [9]:
def td_value_prediction(env, policy):
    alpha = 5e-3
    
    value_vector = np.zeros([len(env.state_space)])
    
    for loop_count in range(1000):
        done = False
        step_count = 0
        s = env.reset()
        
        # Generate Episode
        while not done:
            i_s = env.get_state_index(env.state_space, s)
            pi_s = policy[i_s]
            a = np.random.choice(env.action_space, p=pi_s)
            r, s_next, done = env.step(a)
            
            # td = r + gamma*V'(s)
            i_s_next = env.get_state_index(env.state_space, s_next)
            td = r + env.gamma * value_vector[i_s_next]
            value_vector[i_s] = value_vector[i_s] + alpha * (td - value_vector[i_s])
            
            if done:
                value_vector[i_s_next] = 0
            
            step_count += 1
            s = s_next
            
        if (loop_count + 1) % 100 == 0:
            print(f"[{loop_count + 1}] value_vector: \n{value_vector}")
    return value_vector

In [10]:
env = GridWorld()

# Policy 초기화
policy = list()
for i_s, s in enumerate(env.state_space):
    pi = np.array([0.25]*4)
    policy.append(pi)
policy = np.array(policy)

value_vector = td_value_prediction(env, policy)

value_table = value_vector.reshape(4, 4)

[100] value_vector: 
[4.12259359e-04 2.01733820e-03 1.09899721e-02 3.37261726e-02
 1.51680754e-03 9.41375063e-03 7.00767005e-02 3.18237812e-01
 6.24776370e-03 4.89502998e-02 3.37006687e-01 1.80114861e+00
 1.89597931e-02 2.33142168e-01 1.87978006e+00 0.00000000e+00]
[200] value_vector: 
[0.00974786 0.02537614 0.06562702 0.16145491 0.02631047 0.08104326
 0.28062043 0.70323289 0.0654645  0.26380572 0.97576942 2.76155091
 0.12956582 0.56909882 2.85127675 0.        ]
[300] value_vector: 
[0.04642796 0.08107471 0.18399597 0.32618943 0.09434429 0.20359035
 0.54664561 0.99553795 0.17443978 0.49271728 1.44667124 3.20951537
 0.26961316 1.01142799 3.46156459 0.        ]
[400] value_vector: 
[0.1092599  0.17551616 0.3191544  0.50844427 0.17851246 0.35219157
 0.73057794 1.20569802 0.33321899 0.7628865  1.82403939 3.71201443
 0.45917516 1.28853734 3.50654596 0.        ]
[500] value_vector: 
[0.19145786 0.27825075 0.44953833 0.61787594 0.27964226 0.46250392
 0.87522755 1.35481082 0.4379032  0.9034127

In [11]:
value_table

array([[0.50735668, 0.65905796, 0.90144822, 1.11885034],
       [0.65405153, 0.941402  , 1.50325072, 2.05514418],
       [0.93203573, 1.52457771, 2.84356143, 4.52453327],
       [1.12506053, 2.09368018, 4.58257978, 0.        ]])

# SARSA
* SARSA는 TD Learning의 한 종류로 On-Policy 알고리즘이면서, Action-Value를 사용한다.
* Evaluation + Improvement를 둘 다 수행하는 통합 알고리즘
* Evaluation(TD + $\epsilon-\text{greedy}$)
* (s, a, r, s', a')를 사용한다고 해서 SARSA라고 부른다.
## Evaluation (TD)
* Temporal Difference인데, State or Action Value 중에 Action Value를 Evaluation한다고 보면 된다.
* 그래서, Evaluation 자체는 TD와 같다.
$$ Q(s,a)\leftarrow Q(s,a)+\alpha[r+\gamma Q(s',a')-Q(s,a)] $$
## Evalutation ($\epsilon-\text{greedy}$)
* Episode를 진행하는 동안 초반에는 Agent에게 자율성을 부여하기 위함이다.
$$ \epsilon=\frac{1}{1+kt} $$
* time step($t$)가 진행됨에 따라 Epsilon 값이 바뀌는 것을 이용한다.
* time step이 진행되면 Epsilon이 1에서 0으로 점점 작아진다.
* 이걸 활용하는 방법은 아래와 같다.
$$ \pi(a\mid s) = \begin{cases}argmax_a Q(s,a) &\text{with}\:P(1-\epsilon) \\ Random\:a\in A &\text{with}\:P(\epsilon)\end{cases} $$
* 위 수식에 따르면 Policy가 초반에는 Random한 Action을 할 확률이 높고, Time step이 진행될수록 Action Value가 높은 Action을 할 확률이 높아진다.

In [17]:
def sarsa(env):
    k_alpha = 1e-3
    k_eps = 5e-4
    action_value_matrix = np.zeros([len(env.state_space), len(env.action_space)])

    def sample_action(eps, action_value):
        a_max = action_value.argmax()
        pi = np.zeros([len(env.action_space)])
        pi[:] = eps / len(env.action_space) # eps를 Action 개수만큼 분할해서 eps를 부여한다.
        pi[a_max] = pi[a_max] + 1 - eps
        a = np.random.choice(env.action_space, p=pi)
        return a

    def get_eps(total_step_count): # eps는 Dynamic하게 변동이 되어야 함
        return 1 / (1+k_eps*total_step_count)

    # Repeat sarsa Loop
    total_step_count = 0
    for loop_count in range(10000):
        done = False
        step_count = 0

        s = env.reset()
        i_s = env.get_state_index(env.state_space, s)
        action_value = action_value_matrix[i_s]
        eps = get_eps(total_step_count)
        a = sample_action(eps, action_value) # ??

        # Generate Episode
        while not done:
            r, s_next, done = env.step(a)
            i_s_next = env.get_state_index(env.state_space, s_next)
            action_value_next = action_value_matrix[i_s_next]
            eps = get_eps(total_step_count)
            a_next = sample_action(eps, action_value_next)

            alpha = 1 / (1 + k_alpha * loop_count)
            td = r + env.gamma * action_value_matrix[i_s_next][a_next] - action_value_matrix[i_s][a]
            action_value_matrix[i_s][a] = action_value_matrix[i_s][a] + alpha * td

            if done:
                action_value_matrix[i_s_next] = 0

            step_count += 1
            total_step_count += 1

            s = s_next
            i_s = i_s_next
            a = a_next

        if (loop_count + 1) % 100 == 0:
            print(
                f"[{loop_count + 1}] action_value_matrix: \n{action_value_matrix} "
                + f"eps: {get_eps(total_step_count):.4f}"
                + f"alpha: {alpha:.4f}"
            )

    policy = np.zeros([len(env.state_space), len(env.action_space)])
    return action_value_matrix, policy

In [20]:
env = GridWorld()
action_value_matrix, _ = sarsa(env)

argmax_actions = action_value_matrix.argmax(axis=-1)

value_table = value_vector.reshape(4, 4)
argmax_actions_table = argmax_actions.reshape(4, 4)
print(f"argmax_actions: \n{argmax_actions.reshape(4, 4)}")

[100] action_value_matrix: 
[[ 1.73503815  1.00804815  1.38982674  1.07296228]
 [ 5.49883731  6.34319336  1.56826401  0.45876529]
 [ 0.77654783  0.85774911  1.47929142  0.80535354]
 [ 0.22087599  6.99164588  1.38573531  1.60977844]
 [ 2.37148604  6.0896376   5.07814573  3.38299209]
 [ 2.29904158  1.5716547   3.28007421  3.03558635]
 [ 1.85957573  1.78965544  1.1146205   7.28114334]
 [ 3.31192372  7.15399742  2.36103224  2.35527703]
 [ 1.3736471   4.13648336  2.62541339  5.10471967]
 [ 2.24118499  8.06247851  2.0841921   1.31764629]
 [ 2.87229544  6.26944952  6.39285619  9.        ]
 [ 3.06694236 10.          1.61753295  8.52043512]
 [ 1.77200625  1.70832916  2.53272033  7.55232256]
 [ 2.70494946  5.24395266  4.40890464  9.        ]
 [ 7.69964417  8.98202339  8.04686104 10.        ]
 [ 0.          0.          0.          0.        ]] eps: 0.4641alpha: 0.9099
[200] action_value_matrix: 
[[ 4.73769528  3.04936649  1.94748956  1.86027192]
 [ 1.95917514  4.26813476  3.45727816  5.09542559]
